In [8]:
import picos as pc
import omnipath as op
import pandas as pd

In [9]:
# Filter based on transcriptutorial example using CARNIVAL
#df_op = op.interactions.OmniPath().get()
#df_opf = df_op[(df_op.consensus_direction == 1) & ((df_op.consensus_stimulation == 1) | (df_op.consensus_inhibition == 1))]
#df_opf = df_opf[df_opf.consensus_stimulation != df_opf.consensus_inhibition]
#df_opf["interaction"] = df_opf.consensus_stimulation.astype(int)
#df_opf.loc[df_opf["interaction"] == 0, "interaction"] = -1
#df_opf[["source", "interaction", "target"]]

In [10]:
df_graph = pd.DataFrame(
    {'source': ['I1', 'I2', 'I2', 'P1'],
     'interaction': [1, 1, -1, 1],
     'target': ['P1', 'P1', 'P2', 'T']
    }
)

df_graph

,source,interaction,target
0,I1,1,P1
1,I2,1,P1
2,I2,-1,P2
3,P1,1,T


In [11]:
perturbations = dict(I1=1, I2=1)
measurements = dict(T=1)

In [12]:
# Direct translation of the V2 CARNIVAL ILP implementation with PICOS
def get_edge(row):
    return row.source + "_" + row.target

p = pc.Problem()
vn = dict()
for node in set(df_graph.source) | set(df_graph.target):
    vn[f'nU_{node}'] = pc.BinaryVariable(f'nU_{node}')
    vn[f'nD_{node}'] = pc.BinaryVariable(f'nD_{node}')
    vn[f'nX_{node}'] = pc.IntegerVariable(f'nX_{node}', lower=-1, upper=1)
    vn[f'nAc_{node}'] = pc.IntegerVariable(f'nAc_{node}', lower=-1, upper=1)
    vn[f'nDs_{node}'] = pc.IntegerVariable(f'nDs_{node}', lower=0, upper=100)
    # Add also C8 here
    p.add_constraint(vn[f'nU_{node}'] - vn[f'nD_{node}'] + vn[f'nAc_{node}'] - vn[f'nX_{node}'] == 0)
    
# Create variables for the measurements (to measure a mismatch which goes from 0 to 2)
for k, v in measurements.items():
    vn[f'aD_{k}'] = pc.IntegerVariable(f'aD_{k}', lower=0, upper=2)

# Create the variables for the edges
for row in df_graph.itertuples():
    edge_name = get_edge(row)
    vn[f'eU_{edge_name}'] = pc.BinaryVariable(f'eU_{edge_name}')
    vn[f'eD_{edge_name}'] = pc.BinaryVariable(f'eD_{edge_name}')
    # Constraint C3
    p.add_constraint(vn[f'eU_{edge_name}'] + vn[f'eD_{edge_name}'] <= 1)

# --- Create the set of constraints --- 

# Add constraints for positive and negative measurements (for the objective function)
for k, v in measurements.items():
    if v > 0:
        p.add_constraint(vn[f'nX_{k}'] - vn[f'aD_{k}'] <= 1)
        p.add_constraint(vn[f'nX_{k}'] + vn[f'aD_{k}'] >= 1)
    else:
        p.add_constraint(vn[f'nX_{k}'] - vn[f'aD_{k}'] <= -1)
        p.add_constraint(vn[f'nX_{k}'] + vn[f'aD_{k}'] >= -1)
        
# C1 and C2
for row in df_graph.itertuples():
    # Add constraints for activatory/inhibitory edges edges
    if row.interaction > 0:
        # C1 and C2
        p.add_constraint(vn[f'eU_{get_edge(row)}'] - vn[f'nX_{row.source}'] >= 0)
        p.add_constraint(vn[f'eD_{get_edge(row)}'] + vn[f'nX_{row.source}'] >= 0)
        # C3 and C4
        p.add_constraint(vn[f'eU_{get_edge(row)}'] - vn[f'nX_{row.source}'] - vn[f'eD_{get_edge(row)}'] <= 0)
        p.add_constraint(vn[f'eD_{get_edge(row)}'] + vn[f'nX_{row.source}'] - vn[f'eU_{get_edge(row)}'] <= 0)
    else:
        # C1 and C2
        p.add_constraint(vn[f'eU_{get_edge(row)}'] + vn[f'nX_{row.source}'] >= 0)
        p.add_constraint(vn[f'eD_{get_edge(row)}'] - vn[f'nX_{row.source}'] >= 0)
        # C3 and C4
        p.add_constraint(vn[f'eU_{get_edge(row)}'] + vn[f'nX_{row.source}'] - vn[f'eD_{get_edge(row)}'] <= 0)
        p.add_constraint(vn[f'eD_{get_edge(row)}'] - vn[f'nX_{row.source}'] - vn[f'eU_{get_edge(row)}'] <= 0)
    # Add constraints for loops
    # Basically, find a valid assignment of distances for each node that preserve the order
    # Review this, very intrincated way of preserving order.
    p.add_constraint(101 * vn[f'eU_{get_edge(row)}'] + vn[f'nDs_{row.source}'] - vn[f'nDs_{row.target}'] <= 100)
    p.add_constraint(101 * vn[f'eD_{get_edge(row)}'] + vn[f'nDs_{row.source}'] - vn[f'nDs_{row.target}'] <= 100)
        
# C6 and C7 for incoming edges
for target in df_graph.target.unique():
    eU = [vn[f'eU_{get_edge(row)}'] for row in df_graph[df_graph.target==target].itertuples()]
    if len(eU) > 0:
        # C6
        p.add_constraint(vn[f'nU_{target}'] <= sum(eU))
    eD = [vn[f'eD_{get_edge(row)}'] for row in df_graph[df_graph.target==target].itertuples()]
    if len(eD) > 0:
        # C7
        p.add_constraint(vn[f'nD_{target}'] <= sum(eD))
    
# Add constraints for perturbations
for node in perturbations:
    p.add_constraint(vn[f'nU_{node}'] <= 0)
    p.add_constraint(vn[f'nD_{node}'] <= 0)
    p.add_constraint(vn[f'nX_{node}'] == 1)
    p.add_constraint(vn[f'nX_{node}'] - vn[f'nAc_{node}'] == 0) # looks like nAc vars are redundant
    
# C8 for unperturbed nodes
unperturbed = (set(df_graph.source) | set(df_graph.target)) - set(perturbations.keys())
for node in unperturbed:
    p.add_constraint(vn[f'nAc_{node}'] == 0)


# Add objective function
# Minimize the discrepancies of the measurements (aD vars)
# Use a penalty on the number of active nodes
# NOTE: This is an inneficient way of encoding the constraints with PICOS
penalty = 0.2
obj1 = sum([vn[f'aD_{k}'] for k, v in measurements.items()])
obj2u = penalty * sum([vn[f'nU_{node}'] for node in set(df_graph.source) | set(df_graph.target)])
obj2d = penalty * sum([vn[f'nD_{node}'] for node in set(df_graph.source) | set(df_graph.target)])
p.set_objective('min', sum(obj1 + obj2u + obj2d))
p

<Integer Linear Program>

In [13]:
sol = p.solve()
print(sol.value)
sol

0.4


<feasible primal solution (claimed optimal) from gurobi>

In [14]:
for k, v in vn.items():
    print(k, v.value)

nU_I1 0.0
nD_I1 0.0
nX_I1 1.0
nAc_I1 1.0
nDs_I1 0.0
nU_P2 0.0
nD_P2 0.0
nX_P2 0.0
nAc_P2 0.0
nDs_P2 100.0
nU_T 1.0
nD_T 0.0
nX_T 1.0
nAc_T 0.0
nDs_T 100.0
nU_P1 1.0
nD_P1 0.0
nX_P1 1.0
nAc_P1 0.0
nDs_P1 1.0
nU_I2 0.0
nD_I2 0.0
nX_I2 1.0
nAc_I2 1.0
nDs_I2 0.0
aD_T 0.0
eU_I1_P1 1.0
eD_I1_P1 0.0
eU_I2_P1 1.0
eD_I2_P1 0.0
eU_I2_P2 0.0
eD_I2_P2 1.0
eU_P1_T 1.0
eD_P1_T 0.0
